In [ ]:
!pip install --upgrade pip
# !pip uninstall -y nncase
!pip install nncase
!pip install nncase-kpu

# nncase-2.x need dotnet-7
!sudo apt-get install -y dotnet-sdk-7.0
import _nncase
print(_nncase.__version__)

In [ ]:
# export nncase lib path into PATH
%env PATH=$PATH:/usr/local/lib/python3.8/site-packages

In [ ]:
import nncase
from nncase_base_func import *

def compile_kmodel(model_path, dump_path, calib_data):
    """
    Set compile options and ptq options.
    Compile kmodel.
    Dump the compile-time result to 'compile_options.dump_dir'
    """
    print("\n----------   compile    ----------")
    print("Simplify...")
    model_file = model_simplify(model_path)

    print("Set options...")
    # import_options
    import_options = nncase.ImportOptions()
    
    # compile_options
    compile_options = nncase.CompileOptions()
    compile_options.target = "k230"  # "cpu"
    compile_options.dump_ir = False  # if False, will not dump the compile-time result.
    compile_options.dump_asm = True
    compile_options.dump_dir = dump_path
    
    # preprocess args
    compile_options.preprocess = False
    if compile_options.preprocess:
        compile_options.input_type = "uint8" # "uint8" "float32"
        compile_options.input_shape = [1,224,320,3]
        compile_options.input_range = [0,1]
        compile_options.input_layout = "NHWC" # "NHWC"
        compile_options.swapRB = False
        compile_options.mean = [0,0,0]
        compile_options.std = [1,1,1]
        compile_options.letterbox_value = 0
        compile_options.output_layout = "NHWC" # "NHWC"
    
    # quant
    ptq_options = nncase.PTQTensorOptions()
    
    ptq_options.quant_type = "uint8" # datatype : "float32", "int8", "int16"
    ptq_options.w_quant_type = "uint8"  # datatype : "float32", "int8", "int16"
    ptq_options.calibrate_method = "NoClip" # "kld"
    ptq_options.finetune_weights_method = "NoFineTuneWeights"
    ptq_options.dump_quant_error = False
    ptq_options.dump_quant_error_symmetric_for_signed = False
    
    # detail in docs/MixQuant.md
    ptq_options.quant_scheme = ""
    ptq_options.export_quant_scheme = False
    ptq_options.export_weight_range_by_channel = False
    
    ptq_options.samples_count = len(calib_data[0])
    ptq_options.set_tensor_data(calib_data)

    
    print("Compiling...")
    compiler = nncase.Compiler(compile_options)
    # import
    model_content = read_model_file(model_file)
    if model_path.split(".")[-1] == "onnx":
        compiler.import_onnx(model_content, import_options)
    elif model_path.split(".")[-1] == "tflite":
        compiler.import_tflite(model_content, import_options)
    
    compiler.use_ptq(ptq_options)
    
    # compile
    compiler.compile()
    kmodel = compiler.gencode_tobytes()
    
    kmodel_path = os.path.join(dump_path, "test.kmodel")
    with open(kmodel_path, 'wb') as f:
        f.write(kmodel)
    print("----------------end-----------------")
    return kmodel_path

def run_kmodel(kmodel_path, input_data):
    print("\n---------start run kmodel---------")
    print("Load kmodel...")
    model_sim = nncase.Simulator()
    with open(kmodel_path, 'rb') as f:
        model_sim.load_model(f.read())
    
    print("Set input data...")
    for i, p_d in enumerate(input_data):
        model_sim.set_input_tensor(i, nncase.RuntimeTensor.from_numpy(p_d))
    
    print("Run...")
    model_sim.run()
    
    print("Get output result...")
    all_result = []
    for i in range(model_sim.outputs_size):
        result = model_sim.get_output_tensor(i).to_numpy()
        all_result.append(result)
    print("----------------end-----------------")
    return all_result


In [ ]:
# compile kmodel single input
model_path = "./mbv2.onnx"
dump_path = "./tmp"

# If model has multiple inputs, calib_data format is "[[x1, x2,...], [y1, y2,...], ...]"
# e.g. Model has three inputs (x, y, z), the calib_data is '[[x1, x2, x3],[y1, y2, y3],[z1, z2, z3]]'

calib_data = [[np.random.rand(1, 3, 224, 224).astype(np.float32), np.random.rand(1, 3, 224, 224).astype(np.float32)]]

kmodel_path = compile_kmodel(model_path, dump_path, calib_data)


In [ ]:
# run kmodel(simulate)
import os

kmodel_path = "./tmp/test.kmodel"

# If model has multiple inputs, calib_data format is "[x, y, ...]"
input_data = [np.random.rand(1, 3, 224, 224).astype(np.float32)]

result = run_kmodel(kmodel_path, input_data)

for idx, i in enumerate(result):
    print(i.shape)
    i.tofile(os.path.join(dump_path,"nncase_result_{}.bin".format(idx)))


In [ ]:
# compile kmodel multiple inputs
model_path = "./decoder.onnx"
dump_path = "./tmp_dec"

# If model has multiple inputs, calib_data format is "[[x1, x2,...], [y1, y2,...], ...]"
# e.g. Model has three inputs (x, y, z), the calib_data is '[[x1, x2, x3],[y1, y2, y3],[z1, z2, z3]]'

calib_data = [[ np.random.randint(1, 5, size=[3,100], dtype='int64'), np.random.randint(1, 5, size=[3,100], dtype='int64')],
              [ np.random.rand(100, 3, 192).astype(np.float32), np.random.rand(100, 3, 192).astype(np.float32)],
              [ np.random.rand(3,100).astype(np.float32)>0.5, np.random.rand(3,100).astype(np.float32)>0.5],] # bool

kmodel_path = compile_kmodel(model_path, dump_path, calib_data)



In [ ]:
# run kmodel(simulate)
import os

kmodel_path = "./tmp_dec/test.kmodel"
# If model has multiple inputs, calib_data format is "[x, y, ...]"
input_data = [np.random.randint(1, 5, size=[3,100], dtype='int64'), np.random.rand(100, 3, 192).astype(np.float32), np.random.rand(3,100).astype(np.float32)>0.5,]

result = run_kmodel(kmodel_path, input_data)

for idx, i in enumerate(result):
    print(i.shape)
    i.tofile(os.path.join(dump_path,"nncase_result_{}.bin".format(idx)))
